### IMPORTS

In [68]:
import numpy as np
import pandas as pd
import re
import os

### Source

In [69]:
# https://openscad.org/cheatsheet/

In [70]:


"""
Primitivas

Cube
cube(size = [x,y,z], center = true/false);
cube(size =  x ,     center = true/false);

Sphere
sphere(r = 10);
sphere(d = 2);

Cylinder
cylinder(h = height, r1 = BottomRadius, r2 = TopRadius, center = true/false);

Polyhedron
polyhedron( points = [ [X0, Y0, Z0], [X1, Y1, Z1], ... ], triangles = [ [P0, P1, P2], ... ], convexity = N);   // before 2014.03
polyhedron( points = [ [X0, Y0, Z0], [X1, Y1, Z1], ... ], faces = [ [P0, P1, P2, P3, ...], ... ], convexity = N);   // 2014.03 & later
polyhedron(
  points=[ [10,10,0],[10,-10,0],[-10,-10,0],[-10,10,0], // the four points at base
           [0,0,10]  ],                                 // the apex point 
  faces=[ [0,1,4],[1,2,4],[2,3,4],[3,0,4],              // each triangle side
              [1,0,3],[2,1,3] ]                         // two triangles for square base
 );

 ---

sphere(radius | d=diameter)
cube(size, center)
cube([width,depth,height], center)
cylinder(h,r|d,center)
cylinder(h,r1|d1,r2|d2,center)
polyhedron(points, faces, convexity)
import("….ext", convexity)
linear_extrude(height,center,convexity,twist,slices)
rotate_extrude(angle,convexity)
surface(file = "….ext",center,convexity)


"""

primitive_pattern = ["sphere\\(", "cube\\(", "cylinder\\(", "polyhedron\\(", "linear_extrude\\(", "rotate_extrude\\(", "surface\\("]


In [71]:
"""
Transformations

TRANSLATION
translate([0,0,-5])
{
    cube(10);
    cylinder(r=5,h=10);
}

ROTATION
rotate([45,45,45])
    translate([10,20,30])
        cube(10);

SCALATION
scale(v = [x, y, z]) { ... }

RESIZE
resize(newsize=[30,60,10]) { ... }

MIRROR
mirror(v= [x, y, z] ) { ... } (Normal vector)

OFFSET
offset(delta = r) { ... }

FILL
fill() {
            text(t, 50);
        }

MINKOWSKI
Displays the minkowski sum of child nodes.
minkowski(){...}


HULL
Displays the convex hull of child nodes.
hull() {
    translate([15,10,0]) circle(10);
    circle(10);
}

translate([x,y,z])
rotate([x,y,z])
rotate(a, [x,y,z])
scale([x,y,z])
resize([x,y,z],auto,convexity)
mirror([x,y,z])
multmatrix(m)
color("colorname",alpha)
color("#hexvalue")
color([r,g,b,a])
offset(r|delta,chamfer)
hull()
minkowski(convexity)

transformation_pattern = ["translate\\(", "rotate\\(", "scale\\(", "resize\\(", "mirror\\(",
 "multmatrix\\(", "color\\(", "offset\\(", "hull\\(", "minkowski\\("]

"""

'\nTransformations\n\nTRANSLATION\ntranslate([0,0,-5])\n{\n    cube(10);\n    cylinder(r=5,h=10);\n}\n\nROTATION\nrotate([45,45,45])\n    translate([10,20,30])\n        cube(10);\n\nSCALATION\nscale(v = [x, y, z]) { ... }\n\nRESIZE\nresize(newsize=[30,60,10]) { ... }\n\nMIRROR\nmirror(v= [x, y, z] ) { ... } (Normal vector)\n\nOFFSET\noffset(delta = r) { ... }\n\nFILL\nfill() {\n            text(t, 50);\n        }\n\nMINKOWSKI\nDisplays the minkowski sum of child nodes.\nminkowski(){...}\n\n\nHULL\nDisplays the convex hull of child nodes.\nhull() {\n    translate([15,10,0]) circle(10);\n    circle(10);\n}\n\ntranslate([x,y,z])\nrotate([x,y,z])\nrotate(a, [x,y,z])\nscale([x,y,z])\nresize([x,y,z],auto,convexity)\nmirror([x,y,z])\nmultmatrix(m)\ncolor("colorname",alpha)\ncolor("#hexvalue")\ncolor([r,g,b,a])\noffset(r|delta,chamfer)\nhull()\nminkowski(convexity)\n\ntransformation_pattern = ["translate\\(", "rotate\\(", "scale\\(", "resize\\(", "mirror\\(",\n "multmatrix\\(", "color\\(", "of

In [72]:
"""
Syntax

Syntax
var = value;
var = cond ? value_if_true : value_if_false;
var = function (x) x + x;
module name(…) { … }
name();
function name(…) = …
name();
include <….scad>
use <….scad>
"""

sytnax_petter = ["var = ", "module \w+", "name\\(", "function \w\\(", "include <", "use <"]

In [73]:
"""
Boolean Operations

union()
difference()
intersection()
"""

boolean_pattern = ["union\\(", "difference\\(", "intersection\\("]

In [74]:
"""
Flow Control

for (i = [start:end]) { … }
for (i = [start:step:end]) { … }
for (i = […,…,…]) { … }
for (i = …, j = …, …) { … }
intersection_for(i = [start:end]) { … }
intersection_for(i = [start:step:end]) { … }
intersection_for(i = […,…,…]) { … }
if (…) { … }
let (…) { … }
"""

flow_control_pattern = ["for \\(", "intersection_for\\(i = ", "if \\(", "let \\("]

In [75]:
# Funciones Auxiliares

def analizeText(text,patrones):
    # primitive_pattern + transformation_pattern + sytnax_petter + boolean_pattern + flow_control_pattern +[";"]+saltos_linea

    # 3. Diccionario para almacenar las apariciones de cada patrón
    apariciones = {}

    # 4. Contar las apariciones de cada patrón
    for patron in patrones:
        try:
            # Compilar el patrón para eficiencia
            regex = re.compile(patron)
            
            # Encontrar todas las coincidencias en el texto
            coincidencias = regex.findall(text)
            
            # Guardar la cantidad de coincidencias en el diccionario
            apariciones[patron] = len(coincidencias)
        except re.error as e:
            print(f"Error al compilar el patrón '{patron}': {e}")
    return apariciones

def printResults(result):
    for patron, cantidad in result.items():
        if cantidad > 0:
            print(f"Patrón '{patron}' encontrado {cantidad} veces.")

def getFileNames(folder_path):
    # Ruta de la carpeta que quieres explorar
    carpeta = folder_path

    # Lista para almacenar los nombres de los archivos
    nombres_archivos = []

    # Iterar sobre todos los archivos en la carpeta
    for archivo in os.listdir(carpeta):
        ruta_archivo = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta_archivo):
            nombres_archivos.append(archivo)

    # Imprimir la lista de nombres de archivos
    return nombres_archivos


In [76]:
fileNames = getFileNames("datasets")

# 2. Lista de patrones corregidos a detectar
transformation_pattern = ["translate\\(", "rotate\\(", "scale\\(", "resize\\(", "mirror\\(","multmatrix\\(", "color\\(", "offset\\(", "hull\\(", "minkowski\\("]
primitive_pattern = ["sphere\\(", "cube\\(", "cylinder\\(", "polyhedron\\(", "linear_extrude\\(", "rotate_extrude\\(", "surface\\("]
sytnax_petter = ["var = ", "module \w+", "name\\(", "function \w\\(", "include <", "use <"]
boolean_pattern = ["union\\(", "difference\\(", "intersection\\("]
flow_control_pattern = ["for \\(", "intersection_for\\(i = ", "if \\(", "let \\("]
saltos_linea = [r'\n|\r\n|\r']
# saltos_linea = [r'(?<!//.*)(\r\n|\r|\n)']

patrones = transformation_pattern+primitive_pattern+sytnax_petter+boolean_pattern+flow_control_pattern+saltos_linea+[";"]

resultado_final = {key: np.array([]) for key in patrones}

for file in fileNames:
    with open(f'datasets/{file}', 'r', encoding='utf-8', errors='replace') as file:
        texto = file.read()
        result = analizeText(texto,patrones)
        # printResults(result)
        # print("\n")
        for key in patrones:
            resultado_final[key] = np.append(resultado_final[key],result[key])



In [77]:
for key in patrones:
    print(f"Patrón '{key}'")
    print(pd.DataFrame(resultado_final[key]).describe())

Patrón 'translate\('
                0
count  118.000000
mean    21.211864
std     20.204447
min      0.000000
25%      7.000000
50%     14.000000
75%     30.000000
max    108.000000
Patrón 'rotate\('
                0
count  118.000000
mean     5.500000
std      5.836446
min      0.000000
25%      1.000000
50%      4.000000
75%      8.750000
max     30.000000
Patrón 'scale\('
                0
count  118.000000
mean     1.838983
std      2.785709
min      0.000000
25%      0.000000
50%      1.000000
75%      2.000000
max     16.000000
Patrón 'resize\('
                0
count  118.000000
mean     1.262712
std      4.339105
min      0.000000
25%      0.000000
50%      0.000000
75%      0.000000
max     24.000000
Patrón 'mirror\('
                0
count  118.000000
mean     0.101695
std      0.330496
min      0.000000
25%      0.000000
50%      0.000000
75%      0.000000
max      2.000000
Patrón 'multmatrix\('
           0
count  118.0
mean     0.0
std      0.0
min      0.0
25%      0.

In [78]:
pd.DataFrame(resultado_final["for \\("]).describe()

,0
count,118.000000
mean,0.237288
std,0.854130
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,5.000000
